In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=1,name='make_profiles',walltime='00:20:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 

  

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
#cluster.scale(10) 
cluster.adapt(minimum=1, maximum=20) 


In [3]:
from dask.distributed import Client
client = Client(cluster)


In [4]:
cluster

In [5]:


client



Client Scheduler: tcp://172.30.100.3:44510 Dashboard: http://172.30.100.3:38924/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob

In [7]:
data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'


In [8]:
import Box2x2 as bb


Box_1
Box_2
Box_3
Box_4
Box_5
Box_6
Box_7
Box_8
Box_9
Box_10
Box_11
Box_12
Box_13
Box_14


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


In [31]:
filetyps = {'votemper' : 'gridT', 'vosaline' : 'gridS'}


def compute_mean_profile_1lev_1h(var,date,time,lev):
    filename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_'+filetyps[var]+'_'+date+'-'+date+'.nc'))
    file=filename[0]
    ds=xr.open_dataset(file,chunks={'x':1000,'y':1000})
    data=ds[str(var)][time,lev]
    for ibox in bb.boxes:
        box = ibox
        print(box.name)      
        print(data[box.jmin:box.jmax,box.imin:box.imax].mean(dim={'x','y'}).values)


In [32]:
%time compute_mean_profile_1lev_1h('votemper','20090701',0,0)

Box_1
24.669245
Box_2
24.16464
Box_3
23.792528
Box_4
23.974836
Box_5
23.520323
Box_6
23.462423
Box_7
23.837378
Box_8
24.574602
Box_9
24.804813
Box_10
25.161076
Box_11
25.350632
Box_12
25.072554
Box_13
24.717422
Box_14
24.340858
CPU times: user 259 ms, sys: 28.7 ms, total: 288 ms
Wall time: 1.13 s


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


In [38]:
filetyps = {'votemper' : 'gridT', 'vosaline' : 'gridS'}


def compute_mean_profile_1lev(var,date,lev):
    filename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_'+filetyps[var]+'_'+date+'-'+date+'.nc'))
    file=filename[0]
    ds=xr.open_dataset(file,chunks={'x':1000,'y':1000})
    data=ds[str(var)][:,lev,:,:]
    for ibox in bb.boxes:
        box = ibox
        print(box.name)      
        print(data[:,box.jmin:box.jmax,box.imin:box.imax].mean(dim={'x','y','time_counter'}).values)


In [39]:
%time compute_mean_profile_1lev('votemper','20090701',0)

Box_1
24.612993
Box_2
24.128191
Box_3
23.795946
Box_4
23.99895
Box_5
23.596638
Box_6
23.526333
Box_7
23.879713
Box_8
24.619146
Box_9
24.84057
Box_10
25.131247
Box_11
25.296598
Box_12
24.999277
Box_13
24.629934
Box_14
24.29025
CPU times: user 744 ms, sys: 120 ms, total: 865 ms
Wall time: 10.2 s


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


In [11]:
filetyps = {'votemper' : 'gridT', 'vosaline' : 'gridS'}


def compute_mean_profile(var,date):
    filename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_'+filetyps[var]+'_'+date+'-'+date+'.nc'))
    file=filename[0]
    ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1,'deptht':1})
    data=ds[str(var)]
    for ibox in bb.boxes:
        box = ibox
        print(box.name)      
        print(data[:,:,box.jmin:box.jmax,box.imin:box.imax].mean(dim={'x','y','time_counter'}).values)


In [12]:
%time compute_mean_profile('votemper','20090701')

Box_1
[2.46129932e+01 2.46117821e+01 2.46070518e+01 2.45987358e+01
 2.45890827e+01 2.45790882e+01 2.45692749e+01 2.45596657e+01
 2.45500774e+01 2.45398788e+01 2.45271797e+01 2.45082359e+01
 2.44770470e+01 2.44207821e+01 2.43145561e+01 2.41255608e+01
 2.38546810e+01 2.35534096e+01 2.32560825e+01 2.29714794e+01
 2.26996803e+01 2.24408760e+01 2.21948166e+01 2.19608784e+01
 2.17382736e+01 2.15263577e+01 2.13248444e+01 2.11336918e+01
 2.09527626e+01 2.07818222e+01 2.06206493e+01 2.04687786e+01
 2.03255138e+01 2.01901340e+01 2.00621281e+01 1.99412460e+01
 1.98275394e+01 1.97211819e+01 1.96222134e+01 1.95303936e+01
 1.94452209e+01 1.93660774e+01 1.92923298e+01 1.92230625e+01
 1.91571045e+01 1.90934772e+01 1.90312462e+01 1.89694519e+01
 1.89073029e+01 1.88440380e+01 1.87789116e+01 1.87114086e+01
 1.86415825e+01 1.85700512e+01 1.84977264e+01 1.84255238e+01
 1.83543072e+01 1.82847137e+01 1.82170315e+01 1.81511879e+01
 1.80867233e+01 1.80228653e+01 1.79586143e+01 1.78929119e+01
 1.78246746e+01 1.

[2.3795946e+01 2.3782600e+01 2.3761150e+01 2.3740358e+01 2.3721102e+01
 2.3703331e+01 2.3687399e+01 2.3672846e+01 2.3659155e+01 2.3645857e+01
 2.3632511e+01 2.3618507e+01 2.3602898e+01 2.3584385e+01 2.3560530e+01
 2.3525434e+01 2.3459473e+01 2.3335703e+01 2.3108431e+01 2.2797178e+01
 2.2474064e+01 2.2169952e+01 2.1884171e+01 2.1613085e+01 2.1353956e+01
 2.1111731e+01 2.0886986e+01 2.0684065e+01 2.0501911e+01 2.0338327e+01
 2.0193123e+01 2.0062849e+01 1.9943632e+01 1.9835520e+01 1.9735344e+01
 1.9641581e+01 1.9552631e+01 1.9467485e+01 1.9384270e+01 1.9304832e+01
 1.9227943e+01 1.9153507e+01 1.9081280e+01 1.9010698e+01 1.8940826e+01
 1.8870754e+01 1.8800211e+01 1.8727760e+01 1.8655428e+01 1.8582115e+01
 1.8506929e+01 1.8433073e+01 1.8359568e+01 1.8289928e+01 1.8223490e+01
 1.8160162e+01 1.8099192e+01 1.8039244e+01 1.7978910e+01 1.7916882e+01
 1.7850094e+01 1.7782034e+01 1.7708941e+01 1.7632126e+01 1.7549721e+01
 1.7459791e+01 1.7362356e+01 1.7256414e+01 1.7142118e+01 1.7017776e+01
 1.688

[2.35966358e+01 2.35647869e+01 2.35327301e+01 2.35057945e+01
 2.34826736e+01 2.34627914e+01 2.34458752e+01 2.34312115e+01
 2.34180775e+01 2.34059048e+01 2.33941784e+01 2.33822327e+01
 2.33692341e+01 2.33541374e+01 2.33351917e+01 2.33088760e+01
 2.32670879e+01 2.31912746e+01 2.30433197e+01 2.27892647e+01
 2.24594784e+01 2.21274300e+01 2.18307800e+01 2.15644264e+01
 2.13200378e+01 2.10913448e+01 2.08750095e+01 2.06704636e+01
 2.04789829e+01 2.03021946e+01 2.01408958e+01 1.99949379e+01
 1.98635597e+01 1.97456551e+01 1.96399918e+01 1.95452404e+01
 1.94600716e+01 1.93832531e+01 1.93136292e+01 1.92502098e+01
 1.91919861e+01 1.91379395e+01 1.90871105e+01 1.90386372e+01
 1.89917564e+01 1.89458580e+01 1.89003639e+01 1.88547115e+01
 1.88085041e+01 1.87615433e+01 1.87138481e+01 1.86649494e+01
 1.86138458e+01 1.85600452e+01 1.85035782e+01 1.84447689e+01
 1.83840275e+01 1.83217506e+01 1.82582321e+01 1.81936035e+01
 1.81278000e+01 1.80605392e+01 1.79912739e+01 1.79191933e+01
 1.78433876e+01 1.776301

[2.38797092e+01 2.38590202e+01 2.38361359e+01 2.38176861e+01
 2.38026257e+01 2.37897873e+01 2.37784061e+01 2.37679958e+01
 2.37580967e+01 2.37480431e+01 2.37366943e+01 2.37219906e+01
 2.36994724e+01 2.36574917e+01 2.35722599e+01 2.34202614e+01
 2.31901588e+01 2.28921585e+01 2.25836716e+01 2.22917557e+01
 2.20184689e+01 2.17626839e+01 2.15199051e+01 2.12871189e+01
 2.10639534e+01 2.08520489e+01 2.06531792e+01 2.04683113e+01
 2.02975082e+01 2.01397953e+01 1.99931126e+01 1.98550434e+01
 1.97234745e+01 1.95972195e+01 1.94764729e+01 1.93627739e+01
 1.92578316e+01 1.91621017e+01 1.90745621e+01 1.89930897e+01
 1.89154778e+01 1.88400536e+01 1.87659721e+01 1.86931400e+01
 1.86219234e+01 1.85526009e+01 1.84850521e+01 1.84188576e+01
 1.83536339e+01 1.82890091e+01 1.82245865e+01 1.81599312e+01
 1.80946140e+01 1.80282383e+01 1.79603424e+01 1.78903561e+01
 1.78176880e+01 1.77417107e+01 1.76616707e+01 1.75767555e+01
 1.74861412e+01 1.73891048e+01 1.72850094e+01 1.71732674e+01
 1.70532894e+01 1.692458

[2.51312504e+01 2.51040134e+01 2.50790119e+01 2.50578079e+01
 2.50357742e+01 2.50090389e+01 2.49708290e+01 2.49081974e+01
 2.48007946e+01 2.46350746e+01 2.44259129e+01 2.41985111e+01
 2.39598637e+01 2.37099285e+01 2.34521465e+01 2.31949654e+01
 2.29472179e+01 2.27131157e+01 2.24929180e+01 2.22850819e+01
 2.20877895e+01 2.18997345e+01 2.17203026e+01 2.15492268e+01
 2.13860493e+01 2.12299118e+01 2.10796814e+01 2.09342003e+01
 2.07928391e+01 2.06558132e+01 2.05236568e+01 2.03963318e+01
 2.02725697e+01 2.01502094e+01 2.00264568e+01 1.98986454e+01
 1.97654095e+01 1.96272526e+01 1.94863243e+01 1.93458652e+01
 1.92091446e+01 1.90782318e+01 1.89537163e+01 1.88352489e+01
 1.87221222e+01 1.86137867e+01 1.85102024e+01 1.84114914e+01
 1.83172264e+01 1.82269135e+01 1.81396618e+01 1.80546913e+01
 1.79709377e+01 1.78875179e+01 1.78036690e+01 1.77188301e+01
 1.76323719e+01 1.75435562e+01 1.74515820e+01 1.73558407e+01
 1.72558880e+01 1.71511345e+01 1.70410137e+01 1.69250736e+01
 1.68029289e+01 1.667401

[2.49992828e+01 2.49682293e+01 2.49394684e+01 2.49135380e+01
 2.48849716e+01 2.48489876e+01 2.47972431e+01 2.47177315e+01
 2.45950623e+01 2.44198456e+01 2.42016201e+01 2.39550877e+01
 2.36891556e+01 2.34157810e+01 2.31467686e+01 2.28890209e+01
 2.26456528e+01 2.24166813e+01 2.22004395e+01 2.19947929e+01
 2.17982445e+01 2.16104641e+01 2.14320107e+01 2.12635326e+01
 2.11049862e+01 2.09554329e+01 2.08133221e+01 2.06768436e+01
 2.05443439e+01 2.04143963e+01 2.02857227e+01 2.01583252e+01
 2.00307236e+01 1.99009838e+01 1.97675838e+01 1.96298866e+01
 1.94882469e+01 1.93456554e+01 1.92042484e+01 1.90667095e+01
 1.89360466e+01 1.88149204e+01 1.87027035e+01 1.85994568e+01
 1.85034695e+01 1.84129143e+01 1.83266430e+01 1.82435131e+01
 1.81626205e+01 1.80834332e+01 1.80049248e+01 1.79269905e+01
 1.78489017e+01 1.77697582e+01 1.76888561e+01 1.76055698e+01
 1.75194130e+01 1.74296455e+01 1.73352699e+01 1.72357712e+01
 1.71310425e+01 1.70208187e+01 1.69046021e+01 1.67819901e+01
 1.66523991e+01 1.651581

[2.42902489e+01 2.42414360e+01 2.41977158e+01 2.41520767e+01
 2.41023560e+01 2.40492592e+01 2.39912186e+01 2.39245377e+01
 2.38456516e+01 2.37522202e+01 2.36419201e+01 2.35103817e+01
 2.33502407e+01 2.31592388e+01 2.29468098e+01 2.27246208e+01
 2.25005951e+01 2.22797375e+01 2.20648289e+01 2.18571472e+01
 2.16569405e+01 2.14641228e+01 2.12787571e+01 2.11012745e+01
 2.09321384e+01 2.07715778e+01 2.06193943e+01 2.04748554e+01
 2.03367443e+01 2.02035427e+01 2.00750713e+01 1.99492817e+01
 1.98248062e+01 1.97001324e+01 1.95748138e+01 1.94479389e+01
 1.93219719e+01 1.91965885e+01 1.90756950e+01 1.89595585e+01
 1.88505211e+01 1.87478962e+01 1.86515675e+01 1.85607471e+01
 1.84746113e+01 1.83920822e+01 1.83124676e+01 1.82344055e+01
 1.81573238e+01 1.80797710e+01 1.80009212e+01 1.79209538e+01
 1.78395405e+01 1.77560349e+01 1.76707592e+01 1.75827732e+01
 1.74916000e+01 1.73971424e+01 1.72983856e+01 1.71955814e+01
 1.70894432e+01 1.69800072e+01 1.68643856e+01 1.67422256e+01
 1.66152973e+01 1.648158

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


In [9]:
import numba
from numba import jit

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


In [10]:
box_names=[]
box_jmin=[]
box_jmax=[]
box_imin=[]
box_imax=[]

for ibox in bb.boxes:
    box_names.append(ibox.name)
    box_jmin.append(ibox.jmin)
    box_jmax.append(ibox.jmax)
    box_imin.append(ibox.imin)
    box_imax.append(ibox.imax)

filetyps = {'votemper' : 'gridT', 'vosaline' : 'gridS'}



In [11]:
@jit(nopython=True)
def loop_on_box(data):
    for k in np.arange(1,15,1):
        print(box_names[k])      
        print(np.mean(data[:,:,box_jmin[k]:box_jmax[k],box_imin[k]:box_imax[k]],dim={'x','y','time_counter'}).values)


def compute_mean_profile(var,date):
    filename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_'+filetyps[var]+'_'+date+'-'+date+'.nc'))
    file=filename[0]
    ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1,'deptht':1})
    data=ds[str(var)]
    datan=np.asarray(data)
    loop_on_box(datan)
    

In [ ]:
%time compute_mean_profile('votemper','20090701')

distributed.utils - ERROR - [Errno 12] Cannot allocate memory
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask_jobqueue/deploy/cluster_manager.py", line 173, in _scale
    self.scale_up(n)
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask_jobqueue/core.py", line 468, in scale_up
    self.start_workers(n - active_and_pending)
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask_jobqueue/core.py", line 380, in start_workers
    out = self._submit_job(fn)
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask_jobqueue/core.py", line 372, in _submit_job
    return self._call(shlex.split(self.submit_command) + [script_filename])
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3